In [1]:
import torch
import math
from dataset import SatProblemDataSet
from models import MLP, LayerNormBasicLSTMCell, NeuroSAT
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [2]:
def compute_loss(logits, labels, parameters):
    # x = logits, z = labels
    # max(x, 0) - x * z + log(1 + exp(-abs(x)))
    predict_costs = torch.nn.functional.relu(logits) - logits * labels + torch.log1p(torch.exp(-torch.abs(logits)))
    predict_cost = torch.mean(predict_costs)
    l2_cost = torch.zeros([])
    for p in parameters:
        l2_cost += torch.sum(p ** 2) / 2
    return predict_cost + 1e-9 * l2_cost

In [3]:
n = NeuroSAT(5, 3, 3, 128)

In [4]:
ds = SatProblemDataSet("data")
dl = torch.utils.data.DataLoader(ds)

In [5]:
optim = torch.optim.Adam(n.parameters(), lr = 1e-5, weight_decay = 1e-10)
n.train()

NeuroSAT(
  (LC_msg): MLP(
    (linears): ModuleList(
      (0): Linear(in_features=5, out_features=5, bias=True)
      (1): Linear(in_features=5, out_features=5, bias=True)
      (2): Linear(in_features=5, out_features=5, bias=True)
      (3): Linear(in_features=5, out_features=5, bias=True)
    )
    (activation_func): ReLU()
  )
  (CL_msg): MLP(
    (linears): ModuleList(
      (0): Linear(in_features=5, out_features=5, bias=True)
      (1): Linear(in_features=5, out_features=5, bias=True)
      (2): Linear(in_features=5, out_features=5, bias=True)
      (3): Linear(in_features=5, out_features=5, bias=True)
    )
    (activation_func): ReLU()
  )
  (L_update): LayerNormBasicLSTMCell(
    (fiou_linear): Linear(in_features=15, out_features=20, bias=False)
    (fiou_ln_layers): ModuleList(
      (0): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
      (1): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
      (2): LayerNorm((5,), eps=1e-05, elementwise_affine=True)
      (3):

In [6]:
losses = []
loss_fn = torch.nn.BCELoss()
for _ in range(1):
    for prob, target in dl:
        optim.zero_grad()
        outputs = n(prob[0].float())
    #     print(outputs, prob[1])
        torch.nn.utils.clip_grad_norm_(n.parameters(), 0.5)
#         loss = compute_loss(outputs, target[0], n.parameters())
        print(outputs, target[0])
        loss = loss_fn(outputs, target[0])
        desc = 'loss: %.4f; ' % (loss.item())
        losses.append(loss.item())
        print(desc)

        loss.backward()
        optim.step()

tensor([0.0363], grad_fn=<AddBackward0>) tensor([0.])
loss: 0.0370; 
tensor([0.0361], grad_fn=<AddBackward0>) tensor([1.])
loss: 3.3218; 
tensor([0.0367], grad_fn=<AddBackward0>) tensor([0.])
loss: 0.0374; 
tensor([0.0368], grad_fn=<AddBackward0>) tensor([1.])
loss: 3.3017; 
tensor([0.0385], grad_fn=<AddBackward0>) tensor([0.])
loss: 0.0393; 
tensor([0.0387], grad_fn=<AddBackward0>) tensor([1.])
loss: 3.2532; 
tensor([0.0382], grad_fn=<AddBackward0>) tensor([0.])
loss: 0.0389; 
tensor([0.0383], grad_fn=<AddBackward0>) tensor([1.])
loss: 3.2617; 
tensor([0.0397], grad_fn=<AddBackward0>) tensor([0.])
loss: 0.0405; 
tensor([0.0398], grad_fn=<AddBackward0>) tensor([1.])
loss: 3.2235; 
tensor([0.0394], grad_fn=<AddBackward0>) tensor([0.])
loss: 0.0402; 
tensor([0.0395], grad_fn=<AddBackward0>) tensor([1.])
loss: 3.2316; 
tensor([0.0399], grad_fn=<AddBackward0>) tensor([0.])
loss: 0.0407; 
tensor([0.0400], grad_fn=<AddBackward0>) tensor([1.])
loss: 3.2190; 
tensor([0.0408], grad_fn=<AddBackw

KeyboardInterrupt: 

In [ ]:
for _ in range(1):
    for prob, target in dl:
        optim.zero_grad()
        outputs = n(prob[0].float())
    #     print(outputs, prob[1])
        torch.nn.utils.clip_grad_norm_(n.parameters(), 0.5)
#         loss = compute_loss(outputs, target[0], n.parameters())
        print(outputs, target[0])
        loss = loss_fn(outputs, target[0])
        desc = 'loss: %.4f; ' % (loss.item())
        losses.append(loss.item())
        print(desc)

        loss.backward()
        optim.step()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot([i for i in range(len(losses))], losses)